# Identifying Food Deserts in Roanoke VA
### Hypothesis: do food deserts exist within Roanoke City? 

# CODE SECTION

In [1]:
# import all the necessary libraries
import numpy as np 
import pandas as pd 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans 
!pip install BeautifulSoup4 
from bs4 import BeautifulSoup
!pip install GeoPy
from geopy.geocoders import Nominatim  
import folium 
from pandas.io.json import json_normalize 
print("Libraries imported.")

Libraries imported.


In [2]:
# fetching page data using requests
url = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Roanoke,_Virginia"
data = requests.get(url).text

# parsing data using BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

#listing neighborhood names
table_contents=[]
for hit in soup.find_all("b",limit=50):
    hit=hit.text.strip()
    print(hit)
    



list of neighborhoods in Roanoke city, Virginia
Airport
Belmont
Cherry Hill
Downtown
Edgewood-Summit Hills
Fairland
Fallon
Franklin-Colonial
Gainsboro
Garden City
Gilmer
Grandin Court
Greater Deyerle
Harrison
Hollins
Hurt Park
Kenwood
Loudon-Melrose
Mecca Gardens
Melrose-Rugby
Mill Mountain
Miller Court/Arrowood
Monterey
Morningside
Mountain View
Norwich
Old Southwest
Peachtree/Norwood
Preston Park
Raleigh Court
Ridgewood Park
Riverdale
Riverland/Walnut Hills
Roundhill
Shenandoah West
South Jefferson
South Roanoke
South Washington Heights
Southern Hills
Villa Heights
Wasena
Washington Heights
Washington Park
West End
Westview Terrace
Wildwood
Williamson Road
Wilmont
^


In [3]:
#importing my neighborhood location sheet to check against neighborhood list 
df_geo_cor=pd.read_csv("./RoaNeighborhoods.csv")
df_geo_cor

,Neighborhood,Lat,Long
0,Airport,37.320398,-79.978946
1,Belmont,37.269709,-79.939414
2,Cherry Hill,37.275969,-80.015460
3,Downtown,37.270648,-79.947447
4,Edgewood-Summit Hills,37.292079,-80.025461
5,Fairland,37.302635,-79.979627
6,Fallon,37.273000,-79.921918
7,Franklin-Colonial,37.250908,-79.965838
8,Gainsboro,37.278219,-79.946965
9,Garden City,37.231804,-79.937125


In [4]:
#making my Roanoke map with neighborhood markers
address = "Roanoke VA"

geolocator = Nominatim(user_agent="roanoke_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Roanoke City are {}, {}.'.format(latitude, longitude))

map_roanoke = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, neighborhood in zip(
        df_geo_cor['Lat'], 
        df_geo_cor['Long'], 
        df_geo_cor['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='limegreen',
        fill_opacity=0.7,
        parse_html=False).add_to(map_roanoke)  

map_roanoke

The geograpical coordinates of Roanoke City are 37.270973, -79.9414313.


In [5]:
#adding my credentials and my first category search
CLIENT_ID = 'P45TM52CVLJQXJRCXKKQXUAV441XSX3PX0TNVK4XAMHIZWQU' # your Foursquare ID
CLIENT_SECRET = 'AOKQYKKWBP1AQVGL0F2CVERWWYQ0KV3NSQIRT05XYYXHFORO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
categoryId = '4bf58dd8d48988d118951735,52f2ab2ebcbc57f1066b8b46'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Category for Search:'+ categoryId)

Your credentails:
CLIENT_ID: P45TM52CVLJQXJRCXKKQXUAV441XSX3PX0TNVK4XAMHIZWQU
CLIENT_SECRET:AOKQYKKWBP1AQVGL0F2CVERWWYQ0KV3NSQIRT05XYYXHFORO
Category for Search:4bf58dd8d48988d118951735,52f2ab2ebcbc57f1066b8b46


In [6]:
#checking a neighborhood on the list
df_geo_cor.loc[36, 'Neighborhood']


'South Roanoke'

In [7]:
#neighborhood information
neighborhood_latitude = df_geo_cor.loc[36, 'Lat'] # neighborhood latitude value
neighborhood_longitude = df_geo_cor.loc[36, 'Long'] # neighborhood longitude value

neighborhood_name =df_geo_cor.loc[36, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of South Roanoke are 37.24660479999999, -79.9478323.


In [8]:
#api url
LIMIT=100
radius=800
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    categoryId)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=P45TM52CVLJQXJRCXKKQXUAV441XSX3PX0TNVK4XAMHIZWQU&client_secret=AOKQYKKWBP1AQVGL0F2CVERWWYQ0KV3NSQIRT05XYYXHFORO&v=20180605&ll=37.24660479999999,-79.9478323&radius=800&limit=100&categoryId=4bf58dd8d48988d118951735,52f2ab2ebcbc57f1066b8b46'

In [9]:
#query
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b02933bd8b454698cdfeb3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'South Roanoke',
  'headerFullLocation': 'South Roanoke, Roanoke',
  'headerLocationGranularity': 'neighborhood',
  'query': 'grocery store',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 37.2538048072, 'lng': -79.93880438281178},
   'sw': {'lat': 37.23940479279999, 'lng': -79.95686021718822}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bbe45b31416a5936fecf33c',
       'name': "Tinnell's Finer foods",
       'location': {'address': '2205 Crystal Spring Ave SW',
        'lat': 37.24855352836651,
        'lng': -79.9466807

In [10]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Tinnell's Finer foods,Grocery Store,37.248554,-79.946681
1,Crystal Spring Grocery,Grocery Store,37.248432,-79.946915
2,Earth Fare,Grocery Store,37.251988,-79.953649


In [12]:
def getNearbyVenues(names, lat, long, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, lat, long):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# type your answer here
roanoke_stores = getNearbyVenues(names=df_geo_cor['Neighborhood'],
                                   lat=df_geo_cor['Lat'],
                                   long=df_geo_cor['Long']
                                  )

Airport
Belmont
Cherry Hill
Downtown
Edgewood-Summit Hills
Fairland
Fallon
Franklin-Colonial
Gainsboro
Garden City
Gilmer
Grandin Court
Greater Deyerle
Harrison
Hollins
Hurt Park
Kenwood
Loudon-Melrose
Mecca Gardens
Melrose-Rugby
Mill Mountain 
Miller Court/Arrowood
Monterey
Morningside
Mountain View
Norwich
Old Southwest
Peachtree/Norwood
Preston Park
Raleigh Court
Ridgewood Park
Riverdale
Riverland/Walnut Hills 
Roundhill
Shenandoah West
South Jefferson
South Roanoke
South Washington Heights
Southern Hills
Villa Heights
Wasena
Washington Heights 
Washington Park
West End
Westview Terrace
Wildwood
Williamson Road
Wilmont 


In [14]:
print(roanoke_stores.shape)
roanoke_stores

(22, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Belmont,37.269709,-79.939414,Community Grocery,37.266635,-79.943508,Grocery Store
1,Downtown,37.270648,-79.947447,Community Grocery,37.266635,-79.943508,Grocery Store
2,Edgewood-Summit Hills,37.292079,-80.025461,Kroger,37.296498,-80.028887,Grocery Store
3,Edgewood-Summit Hills,37.292079,-80.025461,Kroger Grocery Pickup and Delivery,37.296837,-80.028969,Grocery Store
4,Miller Court/Arrowood,37.317634,-80.004906,Food Lion Grocery Store,37.314287,-80.003432,Supermarket
5,Mountain View,37.269025,-79.972682,Roanoke Natural Foods Co-Op,37.265245,-79.977260,Grocery Store
6,Mountain View,37.269025,-79.972682,Mick Or Mack,37.265603,-79.975661,Grocery Store
7,Preston Park,37.311129,-79.949089,La Esquina Latina,37.310761,-79.949761,Grocery Store
8,Riverdale,37.255137,-79.922681,Food Lion Grocery Store,37.251336,-79.915752,Supermarket
9,Roundhill,37.298652,-79.948681,Oriental Market,37.304660,-79.945015,Grocery Store


In [15]:
roanoke_stores.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belmont,1,1,1,1,1,1
Downtown,1,1,1,1,1,1
Edgewood-Summit Hills,2,2,2,2,2,2
Miller Court/Arrowood,1,1,1,1,1,1
Mountain View,2,2,2,2,2,2
Preston Park,1,1,1,1,1,1
Riverdale,1,1,1,1,1,1
Roundhill,1,1,1,1,1,1
South Roanoke,3,3,3,3,3,3


In [16]:
#one hot encode
roanoke_onehot = pd.get_dummies(roanoke_stores[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
roanoke_onehot['Neighborhood'] = roanoke_stores['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [roanoke_onehot.columns[-1]] + list(roanoke_onehot.columns[:-1])
roanoke_onehot = roanoke_onehot[fixed_columns]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
roanoke_onehot

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Neighborhood,Grocery Store,Supermarket
0,Belmont,1,0
1,Downtown,1,0
2,Edgewood-Summit Hills,1,0
3,Edgewood-Summit Hills,1,0
4,Miller Court/Arrowood,0,1
5,Mountain View,1,0
6,Mountain View,1,0
7,Preston Park,1,0
8,Riverdale,0,1
9,Roundhill,1,0


In [17]:
roa_grp = roanoke_onehot.groupby('Neighborhood').mean().reset_index()
roa_grp

,Neighborhood,Grocery Store,Supermarket
0,Belmont,1,0
1,Downtown,1,0
2,Edgewood-Summit Hills,1,0
3,Miller Court/Arrowood,0,1
4,Mountain View,1,0
5,Preston Park,1,0
6,Riverdale,0,1
7,Roundhill,1,0
8,South Roanoke,1,0
9,South Washington Heights,1,0


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = roa_grp['Neighborhood']

for ind in np.arange(roa_grp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(roa_grp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
0,Belmont,Grocery Store,Supermarket
1,Downtown,Grocery Store,Supermarket
2,Edgewood-Summit Hills,Grocery Store,Supermarket
3,Miller Court/Arrowood,Supermarket,Grocery Store
4,Mountain View,Grocery Store,Supermarket
5,Preston Park,Grocery Store,Supermarket
6,Riverdale,Supermarket,Grocery Store
7,Roundhill,Grocery Store,Supermarket
8,South Roanoke,Grocery Store,Supermarket
9,South Washington Heights,Grocery Store,Supermarket


In [20]:
# set number of clusters
kclusters = 5

roa_grouped_clustering = roa_grp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(roa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

roa_merged = df_geo_cor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
roa_merged = roa_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='inner')

roa_merged.head() # check the last columns!

,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
1,Belmont,37.269709,-79.939414,0,Grocery Store,Supermarket
3,Downtown,37.270648,-79.947447,0,Grocery Store,Supermarket
4,Edgewood-Summit Hills,37.292079,-80.025461,0,Grocery Store,Supermarket
21,Miller Court/Arrowood,37.317634,-80.004906,1,Supermarket,Grocery Store
24,Mountain View,37.269025,-79.972682,0,Grocery Store,Supermarket


In [22]:
roa_merged.dtypes


Neighborhood             object 
Lat                      float64
Long                     float64
Cluster Labels           int32  
1st Most Common Venue    object 
2nd Most Common Venue    object 
dtype: object

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(roa_merged['Lat'], roa_merged['Long'], roa_merged['Neighborhood'], roa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
url = 'http://zipatlas.com/us/va/roanoke/zip-code-comparison/population-below-poverty-level.htm'
tables = pd.read_html(url)

df = tables[11][1:]
df.columns = tables[11].iloc[0]
df=df.drop(columns=["#","National Rank"])
df[['Lat','Long']]=df.Location.str.split(expand=True,)
df=df.drop(columns=["Location"])
df['Lat'] = df['Lat'].str.replace(r',','')
df['% Poverty Level'] = df['% Poverty Level'].str.replace(r'%','')
df[["Lat", "Long","% Poverty Level"]] = df[["Lat", "Long","% Poverty Level"]].apply(pd.to_numeric)
df

#keep markers for poverty levels above 9.2%
df2 = df.drop(df[df['% Poverty Level'] < 9.2].index, inplace = False)
df2

,Zip Code,City,Population,% Poverty Level,Lat,Long
1,24011,"Roanoke, Virginia",75,48.27,37.270198,-79.941523
2,24016,"Roanoke, Virginia",9572,36.60,37.272432,-79.953147
3,24013,"Roanoke, Virginia",7984,35.20,37.266989,-79.921759
4,24017,"Roanoke, Virginia",23236,25.75,37.297647,-79.990966
5,24012,"Roanoke, Virginia",27314,14.38,37.317594,-79.888805


In [29]:
for lat, lng, neighborhood in zip(
        df2['Lat'], 
        df2['Long'], 
        df2['% Poverty Level']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3.5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  

map_clusters